### 0.1 Import Package

In [1]:
import os
from pyprnt import prnt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import numpy as np

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

import torch
import torchvision
from torchsummary import summary

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

device = torch.device("cuda:0")

def set_seed(seed=2023):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(2023)

PyTorch Version:  2.1.0+cu121
Torchvision Version:  0.16.0+cu121


In [2]:
""" User Define function """

# from model.models import Conv4

from model.pretrained_models import select_model
from dataset_dir.datasets import datasetload

from utils.util_functions import name_parser
from utils.util_functions import get_alias

from utils.visualize_tool import visualize_tensor
from utils.visualize_tool import visualize_filters
from utils.visualize_tool import visualize_feature_map
from utils.visualize_tool import visualize_weight_distribution

from utils.get_data import get_feature_from_dataset
from utils.visualize_tool import visualize_feature_distribution
from utils.util_functions import get_umap_embedding
from utils.visualize_tool import visualize_class_activation_images
from utils.visualize_grad import LayerCam
from utils.util_functions import get_info

from utils.get_data import get_performance_df
from utils.visualize_tool import plot_comparison_each_dataset
from utils.visualize_tool import plot_comparison_each_dataset_only_one
from utils.visualize_tool import plot_comparison_each_dataset_only_two
from utils.visualize_tool import visualize_gradXimage

### 0.2. Directory Setting

In [3]:
""" IMPORTANT

RECURSIVE
- if you want visualization all weight in your directory, then RECURSIVE  = TRUE
- else, you want visualization specific log in your directory, then RECURSIVE = FALSE

SAVE
- if you want save all visualization image, then SAVE = TRUE
- else, you don't want save image, then SAVE = FALSE

SHOW
- if you want visualization result in this jupyter kernel, then SHOW = TRUE
- else, you don't want show image in jupyter kernel, then SHOW = FALSE

"""

RECURSIVE = True
SAVE = True
SHOW = False

Specific model setting: 특정한 모델을 확인하기 위해

In [4]:
if not RECURSIVE :
    conf = dict(
        epoch=100,
        lr=0.001,
        stepsize=50,
        gamma=0.1,
        model="resnet18",
        dataset="cub",
        pretrain=False,
        mode="cus"
    )
    MODEL_DIR = "./VISMODEL"

    name = str(conf['model'])+"_"+str(conf['dataset'])+"_"+str(conf['mode'])
    model_path = MODEL_DIR + name +".pt"
    #log_path = LOG_DIR +  name


    print(model_path)
    #print(log_path)

Entire model setting: directory에 저장되어있는 전체 모델을 확인하기 위해

In [5]:
if RECURSIVE :
    MODEL_DIR = "./VISMODEL"
    # LOG_DIR = "./results/best_log18"

    weight_files = [os.path.join(MODEL_DIR, file) for file in os.listdir(MODEL_DIR) if os.path.isfile(os.path.join(MODEL_DIR, file))]
    # log_files = [os.path.join(LOG_DIR, file) for file in os.listdir(LOG_DIR)]

    print(len(weight_files))
    print(weight_files)
    # print(len(log_files))
    # #print(log_files[0])

3
['./VISMODEL\\resnet18_aircraft_GBwevaSS.pt', './VISMODEL\\resnet18_aircraft_GBwevaSSOverfit.pt', './VISMODEL\\resnet18_aircraft_standard.pt']


### 1. Architecture
특정 데이터셋, 모델의 구조를 확인하기 위한 코드. RECURSIVE하게 모든 weight파일에 대해서 동작하지 않습니다.

In [6]:
if not RECURSIVE :
    trainloader, validloader, testloader, num_class = datasetload(conf['dataset'])
    MODEL = select_model(conf['model'], num_class)
    model_state = torch.load(model_path)
    MODEL.load_state_dict(model_state)

#### 1.1 Model Architecture

In [7]:
if not RECURSIVE :
    print(MODEL)

In [8]:
if not RECURSIVE :
    summary(MODEL, (3, 224, 224), device='cpu')

*사용하고자하는 모델의 layer에 대한 정보를 기록합니다. (e.g., Alexnet)

In [9]:
# setting
CLASS_NUM = dict(
    cifar100=100,
    cub=200,
    cars=196,
    aircraft=100,
)

In [10]:
temp_conf = name_parser(weight_files[0])
temp_conf['model'] = temp_conf['model'].split("\\")[-1]
print(temp_conf)

num_class = CLASS_NUM[temp_conf['dataset']]
temp_model = select_model(temp_conf['model'], num_class)
temp_layer_names = [name for name, _ in temp_model.named_modules()]
print(temp_layer_names)

# setting
if temp_conf['model'] == 'resnet18':
    layer_name_list = ['conv1', 'layer1.0.conv2', 'layer1.1.conv2', 'layer2.0.conv2', 'layer2.1.conv2', \
                'layer3.0.conv2', 'layer3.1.conv2', 'layer4.0.conv2', 'layer4.1.conv2']
    model_info = dict(
    conv1=0, conv2=10, conv3=11, conv4=20, conv5=21, conv6=30, conv7=31, conv8=40, conv9=41)   
elif temp_conf['model'] == 'resnet50':
    layer_name_list = ['conv1', 'layer1.0.conv2', 'layer1.1.conv2', 'layer1.2.conv2', 'layer2.0.conv2', 'layer2.1.conv2', \
                'layer2.2.conv2', 'layer2.3.conv2', 'layer3.0.conv2', 'layer3.1.conv2', 'layer3.2.conv2', 'layer3.3.conv2', \
                'layer3.4.conv2', 'layer3.5.conv2', 'layer4.0.conv2', 'layer4.1.conv2', 'layer4.2.conv2']
    model_info = dict(
    conv1=0, conv2=10, conv3=11, conv4=12, conv5=20, conv6=21, conv7=22, conv8=23, conv9=30, conv10=31, conv11=32, conv12=33, conv13=34, conv14=35, conv15=40, conv16=41, conv17=42)   
else: # alexnet
    layer_name_list = ['features.0', 'features.3', 'features.6', 'features.8', 'features.10']
    model_info = dict(
    conv1=0,
    conv2=3,
    conv3=6,
    conv4=8,
    conv5=10)   
print(get_alias(layer_name_list[1], model_info, model=temp_conf['model']))

{'mode': 'GBwevaSS', 'model': 'resnet18', 'dataset': 'aircraft'}
['', 'conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer1.0', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.1', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer2', 'layer2.0', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.downsample', 'layer2.0.downsample.0', 'layer2.0.downsample.1', 'layer2.1', 'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu', 'layer2.1.conv2', 'layer2.1.bn2', 'layer3', 'layer3.0', 'layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.relu', 'layer3.0.conv2', 'layer3.0.bn2', 'layer3.0.downsample', 'layer3.0.downsample.0', 'layer3.0.downsample.1', 'layer3.1', 'layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.relu', 'layer3.1.conv2', 'layer3.1.bn2', 'layer4', 'layer4.0', 'layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.relu', 'layer4.0.conv2', 'layer4.0.bn2', 'layer4.0.downsample', 'layer4

#### 1.2 Dataset Property

In [11]:
if not RECURSIVE :
    print(num_class)
    print(len(trainloader.dataset))
    print(len(validloader.dataset))
    print(len(testloader.dataset))

In [12]:
if not RECURSIVE :
    img_idx = 0
    image = trainloader.dataset[img_idx][0]
    print(image.shape)

    sns.set_style("white")
    visualize_tensor(image)

#### 1.3 Train/Val Best Accuracy save

In [13]:
def get_best_accuracy(path):
    event_acc = EventAccumulator(path)
    event_acc.Reload()

    if 'scalars' in event_acc.Tags():
        scalar_tags = event_acc.Tags()['scalars']
        
        for i, tag in enumerate(scalar_tags):
            values = event_acc.Scalars(tag)
            max_value = max(v.value for v in values)
            min_value = min(v.value for v in values)
            if tag == 'Acc/train':
                best_train_acc = max_value
            elif tag == 'Acc/val':
                best_val_acc = max_value
            elif tag == 'Loss/train':
                best_train_loss = min_value
            elif tag == 'Loss/val':
                best_val_loss = min_value
    
    return best_train_acc, best_val_acc, best_train_loss, best_val_loss

In [14]:
# """ RECURSIVE : Running This!!!"""
# BEST_VALUES = {}

# for log_path in log_files :
#     conf = name_parser(log_path)
#     best_train_acc, best_val_acc, best_train_loss, best_val_loss = get_best_accuracy(log_path)
#     alias = conf["dataset"]+"-"+conf["mode"]
#     BEST_VALUES[alias] = {'best_train_acc': best_train_acc, 'best_val_acc': best_val_acc, 'best_train_loss': best_train_loss, 'best_val_loss': best_val_loss}

# prnt(BEST_VALUES)

#### 1.4 Train Log Print

In [15]:
# def draw_log(path, isSave=False, overlap=True):
#     conf = name_parser(path)
#     event_acc = EventAccumulator(path)
#     event_acc.Reload()

#     if 'scalars' in event_acc.Tags():
#         scalar_tags = event_acc.Tags()['scalars']

#         if not overlap :
#             plt.figure(figsize=(13, 10))
#             sns.set()

#         for i, tag in enumerate(scalar_tags):
#             values = event_acc.Scalars(tag)
#             steps = [v.step for v in values]
#             data = [v.value for v in values]
            
#             df = pd.DataFrame({'Step': steps, tag: data})

#             plt.subplot(2, 2, i + 1)
#             sns.lineplot(x='Step', y=tag, data=df, label=conf['dataset']+"("+conf['freeze']+")" )
#             plt.xlabel('Steps')
#             plt.ylabel(tag)
#             plt.title(f'{tag} over Time')
            
#         plt.tight_layout()
#         if not overlap :
#             plt.show()

# if not RECURSIVE :
#     draw_log(log_path, SAVE)

# else :
#     plt.figure(figsize=(13, 10))
#     sns.set()
#     for log_path in log_files :
#         draw_log(log_path)
#     plt.show()

### 2. Visualization

#### 2.1. cnn Filter visualization
visualize_filters : 모델을 이루는 각각의 convolution 필터의 가중치를 시각화 합니다. (constraint: 1st channel)

In [16]:
if not RECURSIVE:
    layer_name = "features.0"
    visualize_filters(model_info, MODEL, layer_name, ncols=32, nchannel=5, showAll=False)

In [35]:
""" RECURSIVE : Running This!!!"""
if RECURSIVE:
    for weight_path in tqdm(weight_files) :
        conf = name_parser(weight_path)
        conf['model'] = conf['model'].split("\\")[-1]
        num_class = CLASS_NUM[conf['dataset']]

        MODEL = select_model(conf['model'], num_class)
#        conf['mode'] = "Pre_Train"
        model_state = torch.load(weight_path)
        MODEL.load_state_dict(model_state)
            
        for layer_name in layer_name_list :
            visualize_filters(model_info, MODEL, layer_name, conf, save=SAVE, show=SHOW, ncols=32, nchannel=5, showAll=False)
        
        break

  0%|          | 0/3 [00:00<?, ?it/s]

Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisFilter\#layer_conv1_#channel_0.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisFilter\#layer_conv1_#channel_1.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisFilter\#layer_conv1_#channel_2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisFilter\#layer_conv2_#channel_0.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisFilter\#layer_conv2_#channel_1.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisFilter\#layer_conv2_#channel_2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisFilter\#layer_conv2_#channel_3.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisFilter\#layer_conv2_#channel_4.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisFilter\#layer_conv3_#channel_0.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisFilter\#layer_conv3_#channel_1.png
Plot saved to ./results/figure/GBwevaSS\

  0%|          | 0/3 [04:09<?, ?it/s]

Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisFilter\#layer_conv9_#channel_4.png


#### 2.2. Feature map Visualize
visualize_feature_map : 특정 입력 이미지에 대해 모델의 각 레이어를 통과했을 때 feature_map을 시각화 합니다. (constraint: 1st feature map)

In [18]:
if not RECURSIVE :
    layer_name = 'conv2'
    input_image = torch.unsqueeze(image, 0)

    activation = {}
    visualize_feature_map(activation, MODEL, input_image, layer_name, model_info[layer_name])

In [21]:
""" RECURSIVE : Running This!!!"""

if RECURSIVE :
    img_idx = 0
    img_dict = dict(
        cifar100=None,
        cub=None,
        aircraft=None
    )
    for dataset in img_dict:
        trainloader, validloader, testloader, num_class = datasetload(dataset, batch_size=64)
        image = trainloader.dataset[img_idx][0]
        input_image = torch.unsqueeze(image, 0)
        img_dict[dataset] = input_image
    for weight_path in weight_files :
        conf = name_parser(weight_path)
        conf['model'] = conf['model'].split("\\")[-1]
        input_image = img_dict[conf['dataset']]
        MODEL = select_model(conf['model'], CLASS_NUM[conf['dataset']])
        model_state = torch.load(weight_path)
        MODEL.load_state_dict(model_state)
        MODEL.eval()
        for layer_name in layer_name_list:
            activation = {}
            visualize_feature_map(activation, MODEL, input_image, layer_name, conf, save=SAVE, show=SHOW)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisActivation\#layer_conv1.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisActivation\#layer_layer1.0.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisActivation\#layer_layer1.1.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisActivation\#layer_layer2.0.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisActivation\#layer_layer2.1.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisActivation\#layer_layer3.0.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisActivation\#layer_layer3.1.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisActivation\#layer_layer4.0.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisActivation\#

#### 2.3. weight distribution (each layer)
visualize_weight_distribution : 모델을 이루는 각각의 레이어의 가중치 분포를 시각화 합니다.

In [24]:
if not RECURSIVE :
    visualize_weight_distribution(MODEL, violin_sample=5000)

In [25]:
""" RECURSIVE : Running This!!!"""
if RECURSIVE :
    for weight_path in weight_files :
        conf = name_parser(weight_path)
        conf['model'] = conf['model'].split("\\")[-1]
        num_class = CLASS_NUM[conf['dataset']]
        
        MODEL = select_model(conf['model'], num_class)
        model_state = torch.load(weight_path)
        MODEL.load_state_dict(model_state)
        
        visualize_weight_distribution(MODEL, conf, save=SAVE, show=SHOW, violin_sample=3000)

Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisWeightDist\statistics.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\VisWeightDist\violinplot.png
Plot saved to ./results/figure/GBwevaSSOverfit\resnet18\aircraft\VisWeightDist\statistics.png
Plot saved to ./results/figure/GBwevaSSOverfit\resnet18\aircraft\VisWeightDist\violinplot.png
Plot saved to ./results/figure/standard\resnet18\aircraft\VisWeightDist\statistics.png
Plot saved to ./results/figure/standard\resnet18\aircraft\VisWeightDist\violinplot.png


#### 2.4. Dataset Distribution
visualize_feature_distribution: 모델이 판단하는 CLASS와 실제 정답 CLASS의 분포를 비교합니다.

In [26]:
if not RECURSIVE : 
    features, labels, preds = get_feature_from_dataset(MODEL, 1000, testloader, layer_name, model_info[layer_name])
    # print(features.shape)
    # print(labels.shape)
    # print(preds.shape)

    embedding = get_umap_embedding(features)
    #print(embedding.shape)
    
    visualize_feature_distribution(embedding, labels, preds, conf, layer_name, show=SHOW, save=SAVE)

In [27]:
""" RECURSIVE : Running This!!! """ # LAST #TODO
if RECURSIVE :
    for weight_path in weight_files :
        conf = name_parser(weight_path)
        conf['model'] = conf['model'].split("\\")[-1]
        trainloader, validloader, testloader, num_class = datasetload(conf['dataset'], batch_size=64)
        MODEL = select_model(conf['model'], num_class)
        model_state = torch.load(weight_path)
        MODEL.load_state_dict(model_state)
        
        for layer_name in layer_name_list: 
            features, labels, preds = get_feature_from_dataset(MODEL, 1000, testloader, layer_name)
            embedding = get_umap_embedding(features)
            visualize_feature_distribution(embedding, labels, preds, conf, layer_name, num_class=7, show=SHOW, save=SAVE)

Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\FeatureDist\#layer_conv1.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\FeatureDist\#layer_layer1.0.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\FeatureDist\#layer_layer1.1.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\FeatureDist\#layer_layer2.0.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\FeatureDist\#layer_layer2.1.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\FeatureDist\#layer_layer3.0.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\FeatureDist\#layer_layer3.1.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\FeatureDist\#layer_layer4.0.conv2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\FeatureDist\#layer_layer4.1.conv2.png
Plot saved to ./results/figure/GBwevaSSOverfit\resnet18\aircraft\FeatureDist\#layer_conv1.png
Plot saved to ./results/figure/GBwevaSSOverfit\resn

#### 2.5. Gradient
##### 2.5.1 Layer-CAM
LayerCAM [16] is a simple modification of Grad-CAM [3], which can generate reliable class activation maps from different layers.
visualize_class_activation_images: 모델이 이미지를 분류할 떄, 어떤 부분에 집중하여 판단하는지를 함께 표현하여 시각화합니다.

In [11]:
if not RECURSIVE:
    data =  trainloader.dataset[img_idx]
    origin_img, prep_img, target_class = get_info(data)
    layer_number = 12  # [0, 12]

    layer_cam = LayerCam(MODEL, layer_number)
    cam = layer_cam.generate_cam(prep_img, target_class)
    visualize_class_activation_images(origin_img, cam, conf, layer_number)

    sns.set_style("white")
    visualize_tensor(data[0])

In [12]:
""" RECURSIVE : Running This!!! """ #TODO
if RECURSIVE :
    img_idx = 0
    img_dict = dict(
        cifar100=None,
        cub=None,
        aircraft=None
    )
    for dataset in img_dict:
        trainloader, validloader, testloader, num_class = datasetload(dataset, batch_size=64)
        data = trainloader.dataset[img_idx]
        img_dict[dataset] = data
    for weight_path in weight_files :
        conf = name_parser(weight_path)   
        conf['model'] = conf['model'].split("\\")[-1]     
        data = img_dict[conf['dataset']]
        origin_img, prep_img, target_class = get_info(data)
        MODEL = select_model(conf['model'], CLASS_NUM[conf['dataset']])
        model_state = torch.load(weight_path)
        MODEL.load_state_dict(model_state)
        MODEL.eval()
        for idx, layer in enumerate(layer_name_list):
            layer_cam = LayerCam(MODEL, layer)
            cam = layer_cam.generate_cam(prep_img, target_class)
            visualize_class_activation_images(origin_img, cam, conf, idx, show=SHOW, save=SAVE)

Files already downloaded and verified
Files already downloaded and verified
40000 10000 10000
Files already downloaded and verified
Files already downloaded and verified
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\LayerCAM\Heatmap_#layer_0.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\LayerCAM\Grad_On_Image_#layer_0.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\LayerCAM\Grayscale_#layer_0.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\LayerCAM\Heatmap_#layer_1.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\LayerCAM\Grad_On_Image_#layer_1.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\LayerCAM\Grayscale_#layer_1.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\LayerCAM\Heatmap_#layer_2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\LayerCAM\Grad_On_Image_#layer_2.png
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\LayerCAM\Grayscale_#layer_2.png
Plot saved to 

##### 2.5.2 Grad Times Image
Another technique that is proposed is simply multiplying the gradients with the image itself.
visualize_gradXimage: 모델이 주어진 이미지를 볼 때, 계산된 Gradient와 원본이미지의 곱으로 나타낸 그림을 시각화합니다.

In [13]:
if not RECURSIVE:
    data =  trainloader.dataset[img_idx]
    origin_img, prep_img, target_class = get_info(data)

    visualize_gradXimage(prep_img, target_class, MODEL, conf, show=SHOW, save=SAVE)

In [15]:
""" RECURSIVE : Running This!!! """ #TODO
if RECURSIVE :
    img_idx = 0
    img_dict = dict(
        cifar100=None,
        cub=None,
        aircraft=None
    )
    for dataset in img_dict:
        trainloader, validloader, testloader, num_class = datasetload(dataset, batch_size=64)
        data = trainloader.dataset[img_idx]
        img_dict[dataset] = data
    for weight_path in weight_files :
        conf = name_parser(weight_path)       
        conf['model'] = conf['model'].split("\\")[-1] 
        data = img_dict[conf['dataset']]
        origin_img, prep_img, target_class = get_info(data)

        MODEL = select_model(conf['model'], CLASS_NUM[conf['dataset']])
        model_state = torch.load(weight_path)
        MODEL.load_state_dict(model_state)
        MODEL.eval()

        visualize_gradXimage(prep_img, target_class, MODEL, conf, show=SHOW, save=SAVE)

Files already downloaded and verified
Files already downloaded and verified
40000 10000 10000
Files already downloaded and verified
Files already downloaded and verified
Plot saved to ./results/figure/GBwevaSS\resnet18\aircraft\GradXImage\vanila_grad.png
Plot saved to ./results/figure/GBwevaSSOverfit\resnet18\aircraft\GradXImage\vanila_grad.png
Plot saved to ./results/figure/standard\resnet18\aircraft\GradXImage\vanila_grad.png


### 3. Comparison
[!attention] 아래의 그래프를 그리기 전에, 반드시 1.3.을 사용하여 accuracy결과를 저장해야합니다.

In [ ]:
# df_dataset, df_freezing = get_performance_df(BEST_VALUES)

# for group_key, df in df_dataset.items():
#     print(f"Group: {group_key}")
#     print(df)
#     print()
# for group_key, df in df_freezing.items():
#     print(f"Group: {group_key}")
#     print(df)
#     print()

# print("01100 freezing 설정을 사용하여 cifar100 데이터셋으로 파인튜닝한 모델의 값에 접근")
# group_df = df_dataset['cifar100']
# print(group_df.loc['best_train_acc', '01100'])
# group_df = df_freezing['01100']
# print(group_df.loc['best_train_acc', 'cifar100'])

#### 3.1. Accuracy / Loss (general)
plot_comparison_each_dataset: 각 데이터셋 마다 layer-freezing 변화에 따른 accuracy, loss를 비교합니다.

In [ ]:
# if not RECURSIVE:
#     plot_comparison_each_dataset(df_dataset)

In [ ]:
# """ RECURSIVE SAVE : Running This!!! """
# if RECURSIVE:
#     plot_comparison_each_dataset(df_dataset, show=SHOW, save=SAVE)

#### 3.2. Accuracy / Loss (one-layer)
plot_comparison_each_dataset_only_one: 각 데이터셋 마다 단 하나의 layer만을 선택하여 학습한 경우 acc / loss를 그립니다.

In [ ]:
# if not RECURSIVE:
#     plot_comparison_each_dataset_only_one(df_dataset)

In [ ]:
# """ RECURSIVE SAVE : Running This!!! """
# if RECURSIVE:
#     plot_comparison_each_dataset_only_one(df_dataset, show=SHOW, save=SAVE)

#### 3.3. Accuracy / Loss (two-layer)
plot_comparison_each_dataset_only_two: 각 데이터셋 마다 2개의 layer만을 선택하여 학습한 경우 acc / loss를 그립니다.

In [ ]:
# if not RECURSIVE:
#     plot_comparison_each_dataset_only_two(df_dataset)

In [ ]:
# """ RECURSIVE SAVE : Running This!!! """
# if RECURSIVE:
#     plot_comparison_each_dataset_only_two(df_dataset, show=SHOW, save=SAVE)